In [12]:
from mtspHeuristic import *
from mtspLocalSearch import *
import random

In [13]:
def insert_in_vector(vector, element, index):
    return vector[:index] + [element] + vector[index:]

insert_in_vector([2,3,4,5],element = 10000,index = 1)

[2, 10000, 3, 4, 5]

In [14]:
test_graph = [[0,2,3,2,3],
                [2,0,2,3,3],
                [3,2,0,3,2],
                [2,3,3,0,2],
                [3,3,2,2,0]]

In [15]:
test_graph = [[0,8,5,10,6,3,13,2,10,6,3,4,9,6,17],
         [8,0,9,6,12,7,4,9,15,9,10,6,5,9,11],
         [5,9,0,10,9,9,7,18,7,4,7,3,11,4,10],
         [10,6,10,0,18,2,10,6,5,4,9,12,8,8,10],
         [6,12,9,18,0,9,10,11,11,12,6,5,6,8,9],
         [3,7,9,2,9,0,6,8,9,10,5,2,7,10,6],
         [13,4,7,10,10,6,0,12,8,3,7,4,10,6,9],
         [2,9,18,6,11,8,12,0,7,3,6,18,17,9,2],
         [10,15,7,5,11,9,8,7,0,10,8,3,14,10,3],
         [6,9,4,4,12,10,3,3,10,0,12,5,9,6,3],
         [3,10,7,9,6,5,7,6,8,12,0,19,8,8,11],
         [4,6,3,12,5,2,4,18,3,5,19,0,12,2,9],
         [9,5,11,8,6,7,10,17,14,9,8,12,0,5,6],
         [6,9,4,8,8,10,6,9,10,6,8,2,5,0,2],
         [17,11,10,10,9,6,9,2,3,3,11,9,6,2,0]]


solutions = [[3, 6, 13, 12, 14, 9, 11, 15, 5],
[8, 4, 7],
[2, 10]]

obj_function = [calc_tour_length(i,test_graph) for i in solutions] # 79, 31, 33
obj_function

[79, 31, 23]

In [16]:
def greedy_insertion_construction(distance_matrix, number_of_salesmen):
    cities_vector = [i for i in range(2,len(distance_matrix)+1)]
    shuffle(cities_vector)

    initial_cities = cities_vector[:number_of_salesmen]
    other_cities = cities_vector[number_of_salesmen:]

    initial_solutions = [[1,i,1] for i in initial_cities]
    route_to_increment_index = 0
    final_city = 0
    city_index_to_insert = 0
    max_distance = float("inf")
    for new_city in other_cities:
        print("Building new solution...")
        for route_index in range(0,len(initial_solutions)): # Para cada rota
            print("In route",route_index)
            route = initial_solutions[route_index]
            print(route)
            #max_distance = float("inf")
            for city_index in range(1,len(route)): # Para cada cidade já inserida na dita rota
                city_in_route = route[city_index]
                city_before_in_route = route[city_index-1]
                insertion_cost = distance_matrix[city_in_route-1][new_city-1] + distance_matrix[new_city-1][city_before_in_route-1] - distance_matrix[city_in_route-1][city_before_in_route-1]
                if city_index == 1:
                    print("First insertion: ", insertion_cost)
                if insertion_cost < max_distance:
                    #print(route_to_increment_index)
                    max_distance = insertion_cost
                    route_to_increment_index = route_index
                    city_index_to_insert = city_index
                    final_city = new_city
                    print("Found cheaper insertion of value",insertion_cost, 'at city',new_city,"in position",city_index_to_insert)
        city_to_insert = final_city
        print("Solution built")
        print("-----------------------------------------------------------------------------")
        #print(route_to_increment_index, city_to_insert, city_index_to_insert,max_distance)
        initial_solutions[route_to_increment_index] = insert_in_vector(initial_solutions[route_to_increment_index], city_to_insert, city_index_to_insert)
        # Reset das variáveis do problema
        route_to_increment_index = 0
        max_distance = float("inf")
    initial_solutions = [item[1:-1] for item in initial_solutions]
    obj_function_vector = objective_function_vector_bf(initial_solutions, distance_matrix)
    return initial_solutions, obj_function_vector

greedy_insertion_construction(test_graph,3)

Building new solution...
In route 0
[1, 5, 1]
First insertion:  8
Found cheaper insertion of value 8 at city 3 in position 1
In route 1
[1, 11, 1]
First insertion:  9
In route 2
[1, 13, 1]
First insertion:  7
Found cheaper insertion of value 7 at city 3 in position 1
Solution built
-----------------------------------------------------------------------------
Building new solution...
In route 0
[1, 5, 1]
First insertion:  3
Found cheaper insertion of value 3 at city 12 in position 1
In route 1
[1, 11, 1]
First insertion:  20
In route 2
[1, 3, 13, 1]
First insertion:  2
Found cheaper insertion of value 2 at city 12 in position 1
Solution built
-----------------------------------------------------------------------------
Building new solution...
In route 0
[1, 5, 1]
First insertion:  22
Found cheaper insertion of value 22 at city 4 in position 1
In route 1
[1, 11, 1]
First insertion:  16
Found cheaper insertion of value 16 at city 4 in position 1
In route 2
[1, 12, 3, 13, 1]
First inserti

([[5], [8, 11], [6, 12, 9, 3, 7, 10, 4, 14, 15, 13, 2]], [12, 11, 58])

In [17]:
def GRASP_construction_probabilistic(distance_matrix, number_of_salesmen, alpha):
    cities_vector = [i for i in range(2,len(distance_matrix)+1)]
    shuffle(cities_vector)

    initial_cities = cities_vector[:number_of_salesmen]
    other_cities = cities_vector[number_of_salesmen:]

    initial_solutions = [[1,i,1] for i in initial_cities]
    route_to_increment_index = 0
    #final_city = 0
    city_index_to_insert = 0
    #max_distance = float("inf")
    for new_city in other_cities: # Para cada cidade que não fora inserida anteriormente...
        insertion_candidates = []
        print("Building new solution...")
        
        # Etapa de pesquisa de soluções possíveis
        for route_index in range(0,len(initial_solutions)): # Para cada rota
            print("In route",route_index)
            route = initial_solutions[route_index]
            print(route)
            #max_distance = float("inf")
            for city_index in range(1,len(route)): # Para cada cidade já inserida na dita rota
                city_in_route = route[city_index]
                city_before_in_route = route[city_index-1]
                insertion_cost = distance_matrix[city_in_route-1][new_city-1] + distance_matrix[new_city-1][city_before_in_route-1] - distance_matrix[city_in_route-1][city_before_in_route-1]
                insertion_candidates.append([insertion_cost, city_index, new_city, route_index])
                
#                if city_index == 1:
#                    print("First insertion: ", insertion_cost)
#                if insertion_cost < max_distance:
#                    #print(route_to_increment_index)
#                    max_distance = insertion_cost
#                    route_to_increment_index = route_index
#                    city_index_to_insert = city_index
#                    final_city = new_city
#                    print("Found cheaper insertion of value",insertion_cost, 'at city',new_city,"in position",city_index_to_insert)        

        # Etapa de filtragem do GRASP
        insertion_candidates.sort(key=lambda x: x[0])
  #      print("Candidates: ", insertion_candidates)
        min_sol_value = insertion_candidates[0][0]
        max_sol_value = insertion_candidates[-1][0]
        gap_in_solutions = max_sol_value - min_sol_value
#        print("Min: %d\nMax: %d\nGap: %d\nalpha*Gap: %d" % (min_sol_value, max_sol_value, gap_in_solutions, gap_in_solutions*alpha))
 #       print("Max solution value possible: ", min_sol_value + gap_in_solutions*alpha)
        insertion_candidates_filtered = []
        for item in insertion_candidates:
            if item[0] <= min_sol_value + gap_in_solutions*alpha:
                print(item, " ", min_sol_value + gap_in_solutions*alpha)
                insertion_candidates_filtered.append(item)
   #     print("Filtered Candidates: ", insertion_candidates_filtered)
        insertion_chosen_solution = random.choice(insertion_candidates_filtered)
        city_index_to_insert = insertion_chosen_solution[1]
        city_to_insert = insertion_chosen_solution[2]
        route_to_increment_index = insertion_chosen_solution[3]
        
        #city_to_insert = chosen_city
        print("Solution built")
        print("-----------------------------------------------------------------------------")
        #print(route_to_increment_index, city_to_insert, city_index_to_insert,max_distance)
        initial_solutions[route_to_increment_index] = insert_in_vector(initial_solutions[route_to_increment_index], city_to_insert, city_index_to_insert)
        # Reset das variáveis do problema
        #insertion_candidates = []
    
    initial_solutions = [item[1:-1] for item in initial_solutions]
    obj_function_vector = objective_function_vector_bf(initial_solutions, distance_matrix)
    return initial_solutions, obj_function_vector


In [18]:
def GRASP_construction_limited(distance_matrix, number_of_salesmen, listlimit = 5):
    cities_vector = [i for i in range(2,len(distance_matrix)+1)]
    shuffle(cities_vector)

    initial_cities = cities_vector[:number_of_salesmen]
    other_cities = cities_vector[number_of_salesmen:]

    initial_solutions = [[1,i,1] for i in initial_cities]
    route_to_increment_index = 0
    #final_city = 0
    city_index_to_insert = 0
    #max_distance = float("inf")
    for new_city in other_cities: # Para cada cidade que não fora inserida anteriormente...
        print("Building new solution...")
        insertion_candidates = []
        
        # Etapa de pesquisa de soluções possíveis
        for route_index in range(0,len(initial_solutions)): # Para cada rota
            print("In route",route_index)
            route = initial_solutions[route_index]
            print(route)
            #max_distance = float("inf")
            for city_index in range(1,len(route)): # Para cada cidade já inserida na dita rota
                city_in_route = route[city_index]
                city_before_in_route = route[city_index-1]
                insertion_cost = distance_matrix[city_in_route-1][new_city-1] + distance_matrix[new_city-1][city_before_in_route-1] - distance_matrix[city_in_route-1][city_before_in_route-1]
                insertion_candidates.append([insertion_cost, city_index, new_city, route_index])
                
#                if city_index == 1:
#                    print("First insertion: ", insertion_cost)
#                if insertion_cost < max_distance:
#                    #print(route_to_increment_index)
#                    max_distance = insertion_cost
#                    route_to_increment_index = route_index
#                    city_index_to_insert = city_index
#                    final_city = new_city
#                    print("Found cheaper insertion of value",insertion_cost, 'at city',new_city,"in position",city_index_to_insert)        

        # Etapa de filtragem do GRASP
        insertion_candidates.sort(key=lambda x: x[0])
  #      print("Candidates: ", insertion_candidates)
        min_sol_value = insertion_candidates[0][0]
        max_sol_value = insertion_candidates[-1][0]
        gap_in_solutions = max_sol_value - min_sol_value
#        print("Min: %d\nMax: %d\nGap: %d\nalpha*Gap: %d" % (min_sol_value, max_sol_value, gap_in_solutions, gap_in_solutions*alpha))
 #       print("Max solution value possible: ", min_sol_value + gap_in_solutions*alpha)
        insertion_candidates_filtered = insertion_candidates[:listlimit]
        print("Filtered Candidates: ", insertion_candidates_filtered)
        insertion_chosen_solution = random.choice(insertion_candidates_filtered)
        city_index_to_insert = insertion_chosen_solution[1]
        city_to_insert = insertion_chosen_solution[2]
        route_to_increment_index = insertion_chosen_solution[3]
        
        #city_to_insert = chosen_city
        print("Solution built")
        print("-----------------------------------------------------------------------------")
        #print(route_to_increment_index, city_to_insert, city_index_to_insert,max_distance)
        initial_solutions[route_to_increment_index] = insert_in_vector(initial_solutions[route_to_increment_index], city_to_insert, city_index_to_insert)
        # Reset das variáveis do problema

    initial_solutions = [item[1:-1] for item in initial_solutions]
    obj_function_vector = objective_function_vector_bf(initial_solutions, distance_matrix)
    return initial_solutions, obj_function_vector

In [19]:
solution_1, obj_function_1 = greedy_insertion_construction(test_graph, 3)
solution_1, obj_function_1, objective_function_vector_bf(solution_1, test_graph)

Building new solution...
In route 0
[1, 14, 1]
First insertion:  12
Found cheaper insertion of value 12 at city 4 in position 1
In route 1
[1, 8, 1]
First insertion:  14
In route 2
[1, 5, 1]
First insertion:  22
Solution built
-----------------------------------------------------------------------------
Building new solution...
In route 0
[1, 4, 14, 1]
First insertion:  17
Found cheaper insertion of value 17 at city 15 in position 1
Found cheaper insertion of value 4 at city 15 in position 2
In route 1
[1, 8, 1]
First insertion:  17
In route 2
[1, 5, 1]
First insertion:  20
Solution built
-----------------------------------------------------------------------------
Building new solution...
In route 0
[1, 4, 15, 14, 1]
First insertion:  7
Found cheaper insertion of value 7 at city 13 in position 1
Found cheaper insertion of value 4 at city 13 in position 2
In route 1
[1, 8, 1]
First insertion:  24
In route 2
[1, 5, 1]
First insertion:  9
Solution built
----------------------------------

([[6, 4, 9, 3, 10, 7, 2, 13, 15, 14, 12], [8], [11, 5]],
 [47, 4, 15],
 [47, 4, 15])

In [20]:
GRASP_construction_probabilistic(test_graph, 3, 0.3)

Building new solution...
In route 0
[1, 2, 1]
In route 1
[1, 12, 1]
In route 2
[1, 8, 1]
[7, 1, 10, 0]   7.0
[7, 2, 10, 0]   7.0
[7, 1, 10, 1]   7.0
[7, 2, 10, 1]   7.0
[7, 1, 10, 2]   7.0
[7, 2, 10, 2]   7.0
Solution built
-----------------------------------------------------------------------------
Building new solution...
In route 0
[1, 2, 10, 1]
In route 1
[1, 12, 1]
In route 2
[1, 8, 1]
[1, 2, 4, 0]   6.1
Solution built
-----------------------------------------------------------------------------
Building new solution...
In route 0
[1, 2, 4, 10, 1]
In route 1
[1, 12, 1]
In route 2
[1, 8, 1]
[4, 1, 14, 1]   6.699999999999999
[4, 2, 14, 1]   6.699999999999999
[6, 4, 14, 0]   6.699999999999999
Solution built
-----------------------------------------------------------------------------
Building new solution...
In route 0
[1, 2, 4, 10, 1]
In route 1
[1, 14, 12, 1]
In route 2
[1, 8, 1]
[3, 4, 3, 0]   8.399999999999999
[3, 1, 3, 1]   8.399999999999999
[4, 3, 3, 1]   8.399999999999999
[5,

([[5, 3, 9, 15, 2, 13, 11, 4, 10], [14, 12, 7, 6], [8]], [68, 21, 4])

In [21]:
GRASP_construction_limited(test_graph, 3, 5)

Building new solution...
In route 0
[1, 5, 1]
In route 1
[1, 10, 1]
In route 2
[1, 8, 1]
Filtered Candidates:  [[3, 1, 12, 0], [3, 2, 12, 0], [3, 1, 12, 1], [3, 2, 12, 1], [20, 1, 12, 2]]
Solution built
-----------------------------------------------------------------------------
Building new solution...
In route 0
[1, 5, 1]
In route 1
[1, 10, 1]
In route 2
[1, 12, 8, 1]
Filtered Candidates:  [[0, 2, 4, 2], [8, 1, 4, 1], [8, 2, 4, 1], [14, 3, 4, 2], [18, 1, 4, 2]]
Solution built
-----------------------------------------------------------------------------
Building new solution...
In route 0
[1, 5, 1]
In route 1
[1, 10, 1]
In route 2
[1, 12, 8, 4, 1]
Filtered Candidates:  [[7, 4, 13, 2], [9, 1, 13, 0], [9, 2, 13, 0], [11, 2, 13, 2], [12, 1, 13, 1]]
Solution built
-----------------------------------------------------------------------------
Building new solution...
In route 0
[1, 13, 5, 1]
In route 1
[1, 10, 1]
In route 2
[1, 12, 8, 4, 1]
Filtered Candidates:  [[-3, 2, 2, 2], [4, 1, 2, 0

([[13, 5], [10, 3, 14], [11, 9, 7, 6, 2, 12, 8, 15, 4]], [21, 20, 78])

In [22]:
test_graph = [[0,8,5,10,6,3,13,2,10,6,3,4,9,6,17],
         [8,0,9,6,12,7,4,9,15,9,10,6,5,9,11],
         [5,9,0,10,9,9,7,18,7,4,7,3,11,4,10],
         [10,6,10,0,18,2,10,6,5,4,9,12,8,8,10],
         [6,12,9,18,0,9,10,11,11,12,6,5,6,8,9],
         [3,7,9,2,9,0,6,8,9,10,5,2,7,10,6],
         [13,4,7,10,10,6,0,12,8,3,7,4,10,6,9],
         [2,9,18,6,11,8,12,0,7,3,6,18,17,9,2],
         [10,15,7,5,11,9,8,7,0,10,8,3,14,10,3],
         [6,9,4,4,12,10,3,3,10,0,12,5,9,6,3],
         [3,10,7,9,6,5,7,6,8,12,0,19,8,8,11],
         [4,6,3,12,5,2,4,18,3,5,19,0,12,2,9],
         [9,5,11,8,6,7,10,17,14,9,8,12,0,5,6],
         [6,9,4,8,8,10,6,9,10,6,8,2,5,0,2],
         [17,11,10,10,9,6,9,2,3,3,11,9,6,2,0]]

solutions = [[1,3,4,5,6,1],[1,1],[1,1]]

point_in_city = 2
point_in_solution = 2

print(test_graph[solutions[0][point_in_solution]-1][point_in_city-1])
print(test_graph[solutions[0][point_in_solution]-1][point_in_city-1])

6
6
